I originally inteded to scrape the official WotC magic.wizards.com site for tournament results but they unfortunately don't make much available outside of very recent events. After some research, tournament data will be scraped from mtgtop8.com, who offer *years* worth of back data. Some individual card data could be scraped here as well, but scryfall.com offers much more robust card data via a convenient API.

interesting note on data gathering: https://www.dailyesports.gg/wizards-of-the-coast-mtg-frank-karsten-stop-publishing-gp-results-win-rates/

In [1]:
import re
import requests
from pprint import pprint
from time import sleep
from bs4 import BeautifulSoup, NavigableString
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

# Connect to MongoDB
client = MongoClient('192.168.0.209', 27017)
db = client['capstone_1']
cards_coll = db['cards']
decks_coll = db['deck_lists']
events_coll = db['events']

# Base URLs for building requests
scryfall_api_url = 'https://api.scryfall.com/{}'  # API docs: https://scryfall.com/docs/api
mtgtop8_url = 'https://www.mtgtop8.com/{}'

# mtgtop8.com format keys for building requests
mt8_format_keys = {
    'vintage': 'VI',
    'legacy': 'LE',
    'modern': 'MO',
    'pioneer': 'PI',
    'historic': 'HI',
    'standard': 'ST',
    'commander': 'EDH',
    'limited': 'LI',
    'pauper': 'PAU',
    'peasant': 'PEA',
    'block': 'BL',
    'extended': 'EX',
    'highlander': 'HIGH',
    'canadian_highlander': 'CHL'
}

def query(link, payload={}):
    """A requests wrapper function"""
    response = requests.get(link, params=payload)
    if response.status_code != 200:
        print('WARNING', response.status_code)
        print(response.content)
    return response

def get_card(name_str):
    """Returns data from Scryfall API on an individual card by name"""
    payload = {'fuzzy': '+'.join(name_str.split())}
    response = query(scryfall_api_url.format('cards/named'), payload)
    return response.json()

def hot_soup(url, payload={}):
    """Makes a steaming bowl of hot soup"""
    response = query(url, payload)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

def gather_archtypes(meta_url):
    """Gathering a list of archtypes among deck strategies for a meta"""
    soup = hot_soup(meta_url)
    archtypes = { strat: [] for strat in ['aggro', 'control', 'combo'] }
    for strat in soup.find_all(class_='Stable')[0].find_all(rowspan=True):  # In this table, only the style type headers use 'rowspan'
        strat_str = strat.contents[0].lower()  # Get corrosponding key for archtypes dict
        item = strat.parent

        # Gather each archtype under each strategy type
        while len(archtypes[strat_str]) < int(strat['rowspan']) - 1:  # Rowspan == number of archtypes under this style
            item = item.next_sibling
            if isinstance(item, NavigableString):
                continue
            if item.a:  # If this sibling has a link, we know it's what we're looking for
                text  = item.a.text
                num_decks = int(item.contents[3].text)
                archtypes[strat_str].append((text, num_decks))
    
    return archtypes

def scrape_events(meta_url, collection):
    """Scrape event pages using selenium"""
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)
    driver.get(meta_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    meta_dropdown = soup.find('select', {'name': 'meta'})  # get drop down selector for meta
    selected_meta = meta_dropdown.find('option', selected=True)  # get current meta
    
    def get_next(driver, class_name):
        """Check if the next button is still valid"""
        try:
            return driver.find_element_by_class_name('Nav_PN')
        except Exception as e:
            return False
    page = 1
    while True:
        print(f'\nScraping event page {page}...')
        next_btn = get_next(driver, 'Nav_PN')
        soup = BeautifulSoup(driver.page_source, 'html.parser')  # make some soup
        
        for event in soup.find_all(class_='Stable')[2].find_all(class_='hover_tr'):  # 10 events list table
            """
                This loop iterates through event table rows, pulling out an ID number,
                the star rating and the date of the event
            """
            link = event.a  # associated hyperlink
            eid = re.search(r"e=(\d+)&", link['href']).group(1)  # unique id number
            stars = event.find(class_='O16').find_all('img')  # star rating / level
            collection.insert_one({
                'id': eid,
                'name': link.text,
                'date': event.find(class_='S10').text,
                'level': 4 if 'bigstar' in stars[0]['src'] else len(stars),
                'link': mtgtop8_url.format(link['href']),
                'meta': selected_meta.text
            })
        
        if next_btn:
            next_btn.click()
            page +=1
            sleep(1)
        else:
            driver.close()
            break

def scrape_top_decks(event_url, event_id, collection):
    """Takes in a url to an event page and scrapes the top decks"""
    soup = hot_soup(event_url)
    decks_table = soup.find_all(class_='Stable')[0]
    # num_players = int(re.search(r"(\d+) players -", decks_table.text).group(1))  This fails sometimes but I haven't found the culprit
    for sib in decks_table.next_siblings:
        if isinstance(sib, NavigableString):
            continue
        if sib.a:
            href = sib.a['href']
            link = mtgtop8_url.format('event' + href)
            mainboard, sideboard, archtype = scrape_decklist(link)
            placement, title, pilot = sib.text.split('\n')[1:-1]
            did = re.search(r"d=(\d+)&", href).group(1)  # unique id number
            deck = {
                'id': did,
                'event_id': event_id,
                'title': title,
                'pilot': pilot,
                'archtype': archtype,
                'placement': placement,
                'mainboard': mainboard,
                'sideboard': sideboard,
                'link': link
            }
            decks_coll.insert_one(deck)

def scrape_decklist(deck_url):
    """Takes in a deck url and returns mainboard and sideboard cards with their quantities"""
    soup = hot_soup(deck_url)
    card_re = re.compile(r"(\d+)\s(.+)")
    deck_table = soup.find_all(class_='Stable')[1]
    deck_headers = deck_table.previous_sibling.previous_sibling.find_all('td')
    archtype = deck_headers[2].text.replace('decks', '')
    cardlist = deck_table.table.find_all('table')
    mainboard, sideboard = [], []
    for row in cardlist.pop().find_all('span'):
        count, card = card_re.search(row.parent.text).groups()
        sideboard.append((int(count), card.strip()))

    for col in cardlist:
        for row in col.find_all('span'):
            count, card = card_re.search(row.parent.text).groups()
            mainboard.append((int(count), card.strip()))

    return mainboard, sideboard, archtype

In [48]:
events_coll.find_one({})

In [42]:
def main():
    form = 'standard'
    meta = 'History - All Worlds'
    soup = hot_soup(mtgtop8_url.format('format'), {'f': mt8_format_keys[form]})
    meta_dropdown = soup.find('select', {'name': 'meta'})  # get drop down selector for meta
    selected_meta = meta_dropdown.find('option', selected=True)  # get current meta
    metas = {opt.text: mtgtop8_url.format(opt['value']) for opt in meta_dropdown.find_all('option')}  # meta URLs
    chosen_meta = metas[meta]  # a meta url will be fed into gather_archtypes() and scrape_events()

    
    
    archtypes = gather_archtypes(chosen_meta)
    events = scrape_events(chosen_meta, events_coll)
    
#     for event in events_coll.find():
#         scrape_top_decks(event['link'], event['id'], deck)
            